In [1]:
import json
import os
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.patches as patches
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected = True) 
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
pd.set_option('max_columns', 1000)
from bokeh.models import Panel, Tabs
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
#import lightgbm as lgb
import plotly.figure_factory as ff
import gc

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import json
from tensorflow.keras.preprocessing import text, sequence
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras import layers 
from keras.layers import Reshape,Concatenate 
from tensorflow.keras.layers import Reshape
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LeakyReLU
#from tensorboardcolab import *
from tensorflow.keras.regularizers import l2  
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate, Masking
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling2D
from tqdm import tqdm
import pickle
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec 
from tqdm import tqdm
import tensorflow as tf
import numpy as np 
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
import tokenization
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate, Masking
from tensorflow.keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, Dropout
from tensorflow.keras.preprocessing import text, sequence
from tqdm import tqdm_notebook as tqdm
import fasttext
import os
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling2D
import json
import gc
import pickle
from tensorflow.keras.preprocessing import text, sequence 
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from tensorflow.keras.models import Model 
from tensorflow.keras.models import load_model
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.python.keras import backend
import datetime
from tensorflow.keras.callbacks import TensorBoard 
import warnings
warnings.filterwarnings("ignore")

In [13]:
l=[1,2,3,'gh','pg']
print(*l)

1 2 3 gh pg


In [28]:
%%time

# question_text and document_text are the inputs.
# bool_long means boolean value which indicates whether the data is short or long q&a.
max_seq_length=200
def predict(doc_text,ques_text,bool_long):
    max_seq_length=200
    
    
    
## SHORT Q&A MODEL
    if not bool_long:
        bert_model = load_model("bertmodel")
        tokenizer = pickle.load(open("tokenizerf.pickel","rb")) 

# converting tokens to ID's
        
        
        def TokenizeAndConvertToIds(text):
            tokens= tokenizer.tokenize(text) # tokenize the reviews
            tokens=tokens[0:(max_seq_length-2)] 
            tokens=['[CLS]',*tokens,'[SEP]'] # adding cls and sep at the end
            masked_array=np.array([1]*len(tokens) + [0]* (max_seq_length-len(tokens))) # masking 
            segment_array=np.array([0]*max_seq_length) 
            if(len(tokens)<max_seq_length): 
                padding=['[PAD]']*(max_seq_length-len(tokens)) # padding
                tokens=[*tokens,*padding] 
            tokentoid=np.array(tokenizer.convert_tokens_to_ids(tokens)) # converting the tokens to id
            return tokentoid,masked_array,segment_array

        #dtext=inputs_df['document_text']
        #qtext=inputs_df['question_text']
        
        
#token_to_id conversion of answer data      

# dtext means document_text
# qtext means question_text
 
        dtext_tokens=[]
        dtext_mask=[]
        dtext_segment=[]

        for i in doc_text: 
            tokentoid,masked_array,segment_array=TokenizeAndConvertToIds(i)
            dtext_tokens.append(tokentoid)
            dtext_mask.append(masked_array)
            dtext_segment.append(segment_array)

        
        dtext_tokens=np.asarray(dtext_tokens)
        dtext_mask=np.asarray(dtext_mask) 
        dtext_segment=np.asarray(dtext_segment) 
        
# X_train_pooled_output of answers      
        dtext_pooled_output=bert_model.predict([dtext_tokens,dtext_mask,dtext_segment])     
        

        tokenizer=pickle.load(open("tokenizerlong2f.pickel","rb"))
        
# token_to_sequences conversion of question data       
        
        def compute_text_and_questions1(ques_text,tokenizer): 
            question_tok = tokenizer.texts_to_sequences(ques_text)
            question_tok = sequence.pad_sequences(question_tok)
            return question_tok

        question_tok=compute_text_and_questions1(ques_text,tokenizer)
        
# loading short model
        
        model1=load_model("shortmodelfinal")
        
# Predicting the test data
        dtext_temp=dtext_pooled_output.reshape(dtext_pooled_output.shape[0],dtext_pooled_output.shape[1],1) 
        predf1=model1.predict([question_tok,dtext_temp])    

        
        indf=[]
        for i in range(len(predf1[0])): 
            arg_max=np.argmax(predf1[2][i])
            indf.append(arg_max) 
            
# Extracting YES,NO and NONE            
        yesnolist=[]
        for i in range(len(indf)):
            if indf[i]==0:
                yesnolist.append('NO')
            elif indf[i]==1:
                yesnolist.append('NONE')
            elif indf[i]==2:
                yesnolist.append('YES')

# If NONE is the answer, then predict indices 
        l3=[]
        if yesnolist[0]=='NONE':

            for i in range(len(ques_text)):
                j=''
                print("Predicted Start index is: ",int(predf1[0][i][0]))
                print("Predicted End index is: ",int(predf1[1][i][0]))        
                print("Question is: ", ques_text[i])
                for k in doc_text[i].split(' ')[int(predf1[0][i][0]):int(predf1[1][i][0])]:
                    j=j+" "+k
                print("Answer is: ",j)
                print(" ",end='\n')
        
# YES or NO
        else:
            print(yesnolist[0])
                

                
                
                
## LONG Q&A MODEL                
    else:
        bert_model = load_model("bertmodel")
        tokenizer = pickle.load(open("tokenizerf.pickel","rb"))
# converting tokens to ID's
        
        
        def TokenizeAndConvertToIds(text):
            tokens= tokenizer.tokenize(text) # tokenize the reviews
            tokens=tokens[0:(max_seq_length-2)] 
            tokens=['[CLS]',*tokens,'[SEP]'] # adding cls and sep at the end
            masked_array=np.array([1]*len(tokens) + [0]* (max_seq_length-len(tokens))) # masking 
            segment_array=np.array([0]*max_seq_length) 
            if(len(tokens)<max_seq_length): 
                padding=['[PAD]']*(max_seq_length-len(tokens)) # padding
                tokens=[*tokens,*padding] 
            tokentoid=np.array(tokenizer.convert_tokens_to_ids(tokens)) # converting the tokens to id
            return tokentoid,masked_array,segment_array


        
        
#token_to_id conversion of answer data      

# dtext means document_text
 
        dtext_tokens=[]
        dtext_mask=[]
        dtext_segment=[]

        for i in doc_text: 
            tokentoid,masked_array,segment_array=TokenizeAndConvertToIds(i)
            dtext_tokens.append(tokentoid)
            dtext_mask.append(masked_array)
            dtext_segment.append(segment_array)

        dtext_tokens=np.asarray(dtext_tokens)
        dtext_mask=np.asarray(dtext_mask) 
        dtext_segment=np.asarray(dtext_segment) 
        
# X_train_pooled_output of answers      
        dtext_pooled_output=bert_model.predict([dtext_tokens,dtext_mask,dtext_segment])     
        

        tokenizer=pickle.load(open("tokenizerlong2f.pickel","rb"))
        
# token_to_sequences conversion of question data       
        
        def compute_text_and_questions1(ques_text,tokenizer): 
            question_tok = tokenizer.texts_to_sequences(ques_text)
            question_tok = sequence.pad_sequences(question_tok)
            return question_tok

        question_tok=compute_text_and_questions1(ques_text,tokenizer)
        


        
# loading long model        
        model1=load_model("longmodelfinal")
     
        dtext_temp=dtext_pooled_output.reshape(dtext_pooled_output.shape[0],dtext_pooled_output.shape[1],1) 
#predicting the test data      
        predf1=model1.predict([question_tok,dtext_temp])    
      
        print(' ',end='\n')

# questions and answers in the range of predicted indices
        j=''
        for i in range(len(doc_text)):
            j=''
            print("Predicted Start index is: ",int(predf1[0][i])) 
            print("Predicted End index is: ",int(predf1[1][i]))        
            print(end='\n')
            print("Question is: ", ques_text[i])
            for k in doc_text[i].split(' ')[int(predf1[0][i][0]):int(predf1[1][i][0])]:
                j=j+" "+k
            print("Answer is: ",j)
            print(" ",end='\n')
        

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 13.1 µs


## Giving input extracted from the below

In [26]:
%%time
#long answer
predict(doc_text,ques_text,True)

 
Predicted Start index is:  1381
Predicted End index is:  2339

Question is:  which is the most common use of opt-in e-mail marketing
Answer is:   than traditional mail , mainly because of the high cost and time required in a traditional mail campaign for producing the artwork , printing , addressing , and mailing . </Li> <Li> Businesses and organizations who send a high volume of emails can use an ESP ( email service provider ) to gather information about the behavior of the recipients . The insights provided by consumer response to email marketing help businesses and organizations understand and make use of consumer behavior . </Li> <Li> Email provides a cost - effective method to test different marketing content , including visual , creative , marketing copy , and multimedia assets . The data gathered by testing in the email channel can then be used across all channels of marketing campaigns , both print and digital . </Li> <Li> Advertisers can reach substantial numbers of email su

In [29]:
%%time
#short answer
predict(doc_text,ques_text,False)

Predicted Start index is:  466
Predicted End index is:  476
Question is:  which is the most common use of opt-in e-mail marketing
Answer is:   hide ) <Ul> <Li> 1 History </Li> <Li> 2 Types
 
Predicted Start index is:  412
Predicted End index is:  420
Question is:  how i.met your mother who is the mother
Answer is:   from fans . </P> <P> An alternate ending
 
CPU times: user 31.9 s, sys: 4 s, total: 35.9 s
Wall time: 31 s


## Choosing input from the data

In [2]:
def read_data(path, sample = True, chunksize = 50000): 
    if sample == True:
        df = [] 
        with open(path, 'rt') as reader:
            for i in range(chunksize):
                df.append(json.loads(reader.readline()))
        df = pd.DataFrame(df)
        print('Our sampled dataset have {} rows and {} columns'.format(df.shape[0], df.shape[1]))
    else:
        df = pd.read_json(path, orient = 'records', lines = True)
        print('Our dataset have {} rows and {} columns'.format(df.shape[0], df.shape[1]))
        gc.collect()
    return df

In [3]:
train1 = read_data('simplified-nq-train.jsonl', sample = True)[:50]

Our sampled dataset have 50000 rows and 6 columns


In [17]:
te=pd.DataFrame(train1,columns=['document_text','question_text'])[:2]
te.head()

,document_text,question_text
0,Email marketing - Wikipedia <H1> Email marketi...,which is the most common use of opt-in e-mail ...
1,The Mother ( How I Met Your Mother ) - wikiped...,how i.met your mother who is the mother


In [5]:
te=te.iloc[35] 

In [6]:
te

document_text    List of Italian musical terms used in English ...
question_text              what is the musical term for fast tempo
Name: 35, dtype: object

## Single Inputs

In [7]:
doc_text=[te['document_text']]
ques_text=[te['question_text']]

## Multiple inputs

In [18]:
doc_text=[]
for i in te['document_text'].values:
    doc_text.append(i)

In [19]:
ques_text=[]
for i in te['question_text'].values:
    ques_text.append(i)